<a href="https://colab.research.google.com/github/fotkarra/datastories_for_emme/blob/main/scraping_naftemporiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

Δημιουργία function για να κάνουμε το αίτημα στο server και να μετατρέπουμε την απόκρισή του σε soup

In [225]:
def get_webpage_soup(url="https://www.naftemporiki.gr/newsroom/"):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }

  response = requests.get("https://www.naftemporiki.gr/newsroom/", headers=headers)
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

Ορισμός της σελίδας των teasers που θέλω να σκραπάρω

In [226]:
page=1

In [227]:
url = f"https://www.naftemporiki.gr/{page}"

Σκραπάρισμα της σελίδας

In [228]:
soup = get_webpage_soup(url)

https://www.naftemporiki.gr/1 was scraped successfully


 Αναζήτηση των blocks των άρθρων στη σούπα

In [229]:
articlesL = soup.find_all("div", {"class":"item item-stream position-relative"})
print(len(articlesL))

30


Επιλογή και επισκόπηση της html του πρώτου άρθρου

In [230]:
article = articlesL[0]
print(article.prettify())

<div class="item item-stream position-relative">
 <div class="grid">
  <div class="g-col-3 order-md-0 order-1">
   <div class="item-image mb-2">
    <a href="https://www.naftemporiki.gr/sports/1952861/o-kein-kerdise-ton-proto-titlo-toy-kai-paredose-epiteloys-ton-moytzoyri/">
     <div class="ratio ratio-16x10 d-md-block d-none">
      <img alt="" class="attachment-medium size-medium" decoding="async" fetchpriority="high" height="405" sizes="(max-width: 720px) 100vw, 720px" src="https://www.naftemporiki.gr/wp-content/uploads/2023/11/w04-212955rtewrterte.jpg" srcset="https://www.naftemporiki.gr/wp-content/uploads/2023/11/w04-212955rtewrterte.jpg 720w, https://www.naftemporiki.gr/wp-content/uploads/2023/11/w04-212955rtewrterte-696x392.jpg 696w" width="720"/>
     </div>
     <div class="ratio ratio-3x2 d-md-none">
      <img alt="" class="attachment-medium size-medium" decoding="async" height="405" sizes="(max-width: 720px) 100vw, 720px" src="https://www.naftemporiki.gr/wp-content/uploads

Εντοπισμός των πληροφοριών που θέλουμε κείμενο συνδέσμου  
~tag και το class (αν χρειάζεται) για το κάθε element που θέλουμε (title, time, author κλπ.)
Γράφουμε τον κώδικα για κάθε element και τσεκάρουμε ότι το output είναι αυτό που πρέπει

In [231]:
title = article.find("h3", {"class": "item-title"}).text.strip()
title

'Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσε -επιτέλους- τον «μουτζούρη»'

In [232]:
url = article.find("h3", {"class": "item-title"}).find('a')["href"]  # Find the <a> tag within the <h3> tag
url

'https://www.naftemporiki.gr/sports/1952861/o-kein-kerdise-ton-proto-titlo-toy-kai-paredose-epiteloys-ton-moytzoyri/'

In [233]:
time = article.find("time", {"class":"item-published"})
print(time)

<time class="item-published" datetime="2025-05-06T14:19:41+03:00">06/05/2025 - 14:19</time>


In [234]:
datetime_str = article.find("time")["datetime"]
datetime_str

'2025-05-06T14:19:41+03:00'

In [235]:
# Ημερομηνία
date_str = datetime_str.split("T")[0]
date_str

'2025-05-06'

In [236]:
# Ώρα
time_str = datetime_str.split("T")[1].split("+")[0]
time_str

'14:19:41'

In [237]:
# Συντάκτης
author = article.find("span", {"class":"category fw-bold text-primary"}).text.strip()
author

'Αθλητικά'

Βάζουμε τον κώδικα εξόρυξης των διαφόρων στοιχείων που γράψαμε παραπάνω μέσα σε ένα function

In [238]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    try:
      title = article.find("h3", {"class": "item-title"}).text.strip()
    except:
      title=""
    try:
      article_url = article.find("h3", {"class": "item-title"}).find("a")["href"]
    except:
      article_url=""
    try:
      time = article.find("time", {"class":"item-published"})

      #datetime_str = article.find("time")["datetime"]
      datetime_str = time.text
      print(datetime_str)
      date_str = datetime_str.split("-")[0]
      print(date_str)
      time_str = datetime_str.split("-")[1]
      print(time_str)

      #print(f"Title: {title}, Date: {date_str}, Time: {time_str}")
    except:
      date_str = ""
      time_str = ""
    try:
      author = article.find("span", {"class":"category fw-bold text-primary"}).text.strip()
    except:
      author = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "author": author}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL


 Έλεγχος καλής λειτουργίας

In [239]:
# Εξόρυξη των πληροφοριών των άρθρων από τη σούπα μιας σελίδας
articles_dataL = get_teaser_elements_from_articlesL(articlesL)
# Δημιουργία dataframe
df = pd.DataFrame(articles_dataL)

06/05/2025 - 14:19
06/05/2025 
 14:19
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:12
06/05/2025 
 14:12
06/05/2025 - 13:59
06/05/2025 
 13:59
06/05/2025 - 13:50
06/05/2025 
 13:50
06/05/2025 - 13:44
06/05/2025 
 13:44
06/05/2025 - 13:43
06/05/2025 
 13:43
06/05/2025 - 13:40
06/05/2025 
 13:40
06/05/2025 - 13:31
06/05/2025 
 13:31
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:26
06/05/2025 
 13:26
06/05/2025 - 13:23
06/05/2025 
 13:23
06/05/2025 - 13:20
06/05/2025 
 13:20
06/05/2025 - 13:11
06/05/2025 
 13:11
06/05/2025 - 13:07
06/05/2025 
 13:07
06/05/2025 - 12:53
06/05/2025 
 12:53
06/05/2025 - 12:52
06/05/2025 
 12:52
06/05/2025 - 12:47
06/05/2025 
 12:47
06/05/2025 - 12:40
06/05/2025 
 12:40
06/05/2025 - 12:30
06/05/2025 
 12:30
06/05/2025 - 12:30
06/05/2025 
 12:30
06/05/2025 - 12:28
06/05/2025 
 12:28
06/05/2025 - 12:27
06/05/2025 
 12:27
06/05/2025 - 12:25
06/05/2025 
 12:25
06/05/2025 -

In [240]:
print(articles_dataL)

[{'title': 'Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσε -επιτέλους- τον «μουτζούρη»', 'article_url': 'https://www.naftemporiki.gr/sports/1952861/o-kein-kerdise-ton-proto-titlo-toy-kai-paredose-epiteloys-ton-moytzoyri/', 'date': '06/05/2025 ', 'time': ' 14:19', 'author': 'Αθλητικά'}, {'title': 'Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη', 'article_url': 'https://www.naftemporiki.gr/sports/1952852/sto-kyrios-tamplo-tis-romis-i-sakkari/', 'date': '06/05/2025 ', 'time': ' 14:15', 'author': 'Αθλητικά'}, {'title': 'Piraeus Bank reports strong profitability in Q1', 'article_url': 'https://www.naftemporiki.gr/english/1952854/piraeus-bank-reports-strong-profitability-in-q1/', 'date': '06/05/2025 ', 'time': ' 14:15', 'author': 'English'}, {'title': 'Χίος: Νεκρός απεγκλωβίστηκε αλλοδαπός από απόκρημνη περιοχή της Αμπέλου', 'article_url': 'https://www.naftemporiki.gr/society/1952848/chios-nekros-apegklovistike-allodapos-apo-apokrimni-periochi-tis-ampeloy/', 'date': '06/05/2025 ', 'time': ' 14:12', 

In [241]:
df

,title,article_url,date,time,author
0,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:19,Αθλητικά
1,Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη,https://www.naftemporiki.gr/sports/1952852/sto...,06/05/2025,14:15,Αθλητικά
2,Piraeus Bank reports strong profitability in Q1,https://www.naftemporiki.gr/english/1952854/pi...,06/05/2025,14:15,English
3,Χίος: Νεκρός απεγκλωβίστηκε αλλοδαπός από απόκ...,https://www.naftemporiki.gr/society/1952848/ch...,06/05/2025,14:12,Κοινωνία
4,10th Mediterranean Yacht Show in Nafplio,https://www.naftemporiki.gr/english/1952846/10...,06/05/2025,13:59,English
5,ΕΟΤ: Προβολή του ελληνικού βιώσιμου τουρισμού ...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:50,Οικονομία
6,ΤτΕ: «Έκλεισε η ψαλίδα» στα επιτόκια δανείων –...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:44,Οικονομία
7,ΔΕΗ: Ξεκινά η β΄ φάση κατασκευής του φωτοβολτα...,https://www.naftemporiki.gr/business/1952808/d...,06/05/2025,13:43,Επιχειρήσεις
8,Eurovision 2025: Με επιτυχία ολοκληρώθηκε η πρ...,https://www.naftemporiki.gr/culture/music/1952...,06/05/2025,13:40,Μουσική
9,Δασμοί: Η Mattel αυξάνει τις τιμές στις κούκλε...,https://www.naftemporiki.gr/finance/world/1952...,06/05/2025,13:31,Διεθνή


 Δημιουργία κώδικα επανάληψης για σκραπάρισμα πολλών σελίδων από το ίδιο site

In [242]:
def scrape_naftemporiki_teaser_pages(archive_name, start_page_nbr, end_page_nbr):
  # Κενή αρχική λίστα
  all_articles_dataL = []
  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του kathmerini.gr
    url = f"https://www.naftemporiki.gr/{archive_name}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*20, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    articlesL = soup.find_all("div", {"class":"item item-stream position-relative"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

In [243]:
df

,title,article_url,date,time,author
0,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:19,Αθλητικά
1,Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη,https://www.naftemporiki.gr/sports/1952852/sto...,06/05/2025,14:15,Αθλητικά
2,Piraeus Bank reports strong profitability in Q1,https://www.naftemporiki.gr/english/1952854/pi...,06/05/2025,14:15,English
3,Χίος: Νεκρός απεγκλωβίστηκε αλλοδαπός από απόκ...,https://www.naftemporiki.gr/society/1952848/ch...,06/05/2025,14:12,Κοινωνία
4,10th Mediterranean Yacht Show in Nafplio,https://www.naftemporiki.gr/english/1952846/10...,06/05/2025,13:59,English
5,ΕΟΤ: Προβολή του ελληνικού βιώσιμου τουρισμού ...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:50,Οικονομία
6,ΤτΕ: «Έκλεισε η ψαλίδα» στα επιτόκια δανείων –...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:44,Οικονομία
7,ΔΕΗ: Ξεκινά η β΄ φάση κατασκευής του φωτοβολτα...,https://www.naftemporiki.gr/business/1952808/d...,06/05/2025,13:43,Επιχειρήσεις
8,Eurovision 2025: Με επιτυχία ολοκληρώθηκε η πρ...,https://www.naftemporiki.gr/culture/music/1952...,06/05/2025,13:40,Μουσική
9,Δασμοί: Η Mattel αυξάνει τις τιμές στις κούκλε...,https://www.naftemporiki.gr/finance/world/1952...,06/05/2025,13:31,Διεθνή


In [244]:
import pandas as pd


In [245]:
df.describe()

,title,article_url,date,time,author
count,30,30,30,30,30
unique,30,30,1,27,14
top,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:15,Κοινωνία
freq,1,1,30,2,4


Δοκιμή του function σε 2 σελίδες, από τo finance

In [246]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time # Make sure 'time' is imported here
import random

In [250]:
archive_name="society"
naftemporiki_society_df = scrape_naftemporiki_teaser_pages("society", 1, 2)   #και στο society και στο politics μου βγαζει νέα από το newsroom
naftemporiki_society_df

Scraping page 1: https://www.naftemporiki.gr/society/page/1/                    https://www.naftemporiki.gr/society/page/1/ was scraped successfully
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:21
06/05/2025 
 14:21
06/05/2025 - 14:19
06/05/2025 
 14:19
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:12
06/05/2025 
 14:12
06/05/2025 - 13:59
06/05/2025 
 13:59
06/05/2025 - 13:50
06/05/2025 
 13:50
06/05/2025 - 13:44
06/05/2025 
 13:44
06/05/2025 - 13:43
06/05/2025 
 13:43
06/05/2025 - 13:40
06/05/2025 
 13:40
06/05/2025 - 13:31
06/05/2025 
 13:31
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:26
06/05/2025 
 13:26
06/05/2025 - 13:23
06/05/2025 
 13:23
06/05/2025 - 13:20
06/05/2025 
 13:20
06/05/2025 - 13:11
06/05/2025 
 13:11
06/05/2025 - 13:07
06/05/2025 
 13:07
06/05/2025 - 12:53
06/05/2025 
 12:53
06/05/2025 - 12:52
06/05/2025 
 12:52
06/05/2025 - 12

,title,article_url,date,time,author
0,Η ανάρτηση του Κυριάκου Μητσοτάκη για τα 13 χρ...,https://www.naftemporiki.gr/politics/1952863/i...,06/05/2025,14:23,Πολιτική
1,Πολιτική κρίση στη Γερμανία: «Να παραιτηθεί ο ...,https://www.naftemporiki.gr/kosmos/1952864/pol...,06/05/2025,14:23,Κόσμος
2,Επιτήρηση και αγγαρεία….,https://www.naftemporiki.gr/opinion/1952849/ep...,06/05/2025,14:21,Απόψεις
3,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:19,Αθλητικά
4,Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη,https://www.naftemporiki.gr/sports/1952852/sto...,06/05/2025,14:15,Αθλητικά
5,Piraeus Bank reports strong profitability in Q1,https://www.naftemporiki.gr/english/1952854/pi...,06/05/2025,14:15,English
6,Χίος: Νεκρός απεγκλωβίστηκε αλλοδαπός από απόκ...,https://www.naftemporiki.gr/society/1952848/ch...,06/05/2025,14:12,Κοινωνία
7,10th Mediterranean Yacht Show in Nafplio,https://www.naftemporiki.gr/english/1952846/10...,06/05/2025,13:59,English
8,ΕΟΤ: Προβολή του ελληνικού βιώσιμου τουρισμού ...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:50,Οικονομία
9,ΤτΕ: «Έκλεισε η ψαλίδα» στα επιτόκια δανείων –...,https://www.naftemporiki.gr/finance/economy/19...,06/05/2025,13:44,Οικονομία


In [251]:
naftemporiki_politics_df = scrape_naftemporiki_teaser_pages("politics", 1, 2)
naftemporiki_politics_df

Scraping page 1: https://www.naftemporiki.gr/politics/page/1/                    https://www.naftemporiki.gr/politics/page/1/ was scraped successfully
06/05/2025 - 14:39
06/05/2025 
 14:39
06/05/2025 - 14:37
06/05/2025 
 14:37
06/05/2025 - 14:30
06/05/2025 
 14:30
06/05/2025 - 14:27
06/05/2025 
 14:27
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:21
06/05/2025 
 14:21
06/05/2025 - 14:19
06/05/2025 
 14:19
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:12
06/05/2025 
 14:12
06/05/2025 - 13:59
06/05/2025 
 13:59
06/05/2025 - 13:50
06/05/2025 
 13:50
06/05/2025 - 13:44
06/05/2025 
 13:44
06/05/2025 - 13:43
06/05/2025 
 13:43
06/05/2025 - 13:40
06/05/2025 
 13:40
06/05/2025 - 13:31
06/05/2025 
 13:31
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:26
06/05/2025 
 13:26
06/05/2025 - 13:23
06/05/2025 
 13:23
06/05/2025 - 13:20
06/05/2025 
 13:20
06/05/2025 - 

,title,article_url,date,time,author
0,Σαρωνικός: Απαγόρευση θαλάσσιων σπορ και τζετ ...,https://www.naftemporiki.gr/society/1952875/sa...,06/05/2025,14:39,Κοινωνία
1,Αττική: Έφοδος της Αστυνομίας σε κτηματολογικά...,https://www.naftemporiki.gr/society/1952868/at...,06/05/2025,14:37,Κοινωνία
2,Χρηματιστήριο: Τραπεζικές πιέσεις μέσω… Γερμανίας,https://www.naftemporiki.gr/finance/markets/19...,06/05/2025,14:30,Αγορές
3,Σύλληψη εκπαιδευτικού φυγόποινου για γενετήσιε...,https://www.naftemporiki.gr/society/1952865/sy...,06/05/2025,14:27,Κοινωνία
4,Η ανάρτηση του Κυριάκου Μητσοτάκη για τα 13 χρ...,https://www.naftemporiki.gr/politics/1952863/i...,06/05/2025,14:23,Πολιτική
5,Πολιτική κρίση στη Γερμανία: «Να παραιτηθεί ο ...,https://www.naftemporiki.gr/kosmos/1952864/pol...,06/05/2025,14:23,Κόσμος
6,Επιτήρηση και αγγαρεία….,https://www.naftemporiki.gr/opinion/1952849/ep...,06/05/2025,14:21,Απόψεις
7,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:19,Αθλητικά
8,Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη,https://www.naftemporiki.gr/sports/1952852/sto...,06/05/2025,14:15,Αθλητικά
9,Piraeus Bank reports strong profitability in Q1,https://www.naftemporiki.gr/english/1952854/pi...,06/05/2025,14:15,English


In [252]:
naftemporiki_tempi_df = scrape_naftemporiki_teaser_pages("search/τέμπη", 1, 2)        #To ίδιο και εδώ, δεν μπορω να κανω search
naftemporiki_tempi_df

Scraping page 1: https://www.naftemporiki.gr/search/τέμπη/page/1/                    https://www.naftemporiki.gr/search/τέμπη/page/1/ was scraped successfully
06/05/2025 - 14:39
06/05/2025 
 14:39
06/05/2025 - 14:37
06/05/2025 
 14:37
06/05/2025 - 14:30
06/05/2025 
 14:30
06/05/2025 - 14:27
06/05/2025 
 14:27
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:23
06/05/2025 
 14:23
06/05/2025 - 14:21
06/05/2025 
 14:21
06/05/2025 - 14:19
06/05/2025 
 14:19
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:15
06/05/2025 
 14:15
06/05/2025 - 14:12
06/05/2025 
 14:12
06/05/2025 - 13:59
06/05/2025 
 13:59
06/05/2025 - 13:50
06/05/2025 
 13:50
06/05/2025 - 13:44
06/05/2025 
 13:44
06/05/2025 - 13:43
06/05/2025 
 13:43
06/05/2025 - 13:40
06/05/2025 
 13:40
06/05/2025 - 13:31
06/05/2025 
 13:31
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:29
06/05/2025 
 13:29
06/05/2025 - 13:26
06/05/2025 
 13:26
06/05/2025 - 13:23
06/05/2025 
 13:23
06/05/2025 - 13:20
06/05/2025 
 13:20
06/05

,title,article_url,date,time,author
0,Αττική: Απαγόρευση θαλάσσιων σπορ και τζετ σκι...,https://www.naftemporiki.gr/society/1952875/at...,06/05/2025,14:39,Κοινωνία
1,Αττική: Έφοδος της Αστυνομίας σε κτηματολογικά...,https://www.naftemporiki.gr/society/1952868/at...,06/05/2025,14:37,Κοινωνία
2,Χρηματιστήριο: Τραπεζικές πιέσεις μέσω… Γερμανίας,https://www.naftemporiki.gr/finance/markets/19...,06/05/2025,14:30,Αγορές
3,Σύλληψη εκπαιδευτικού φυγόποινου για γενετήσιε...,https://www.naftemporiki.gr/society/1952865/sy...,06/05/2025,14:27,Κοινωνία
4,Η ανάρτηση του Κυριάκου Μητσοτάκη για τα 13 χρ...,https://www.naftemporiki.gr/politics/1952863/i...,06/05/2025,14:23,Πολιτική
5,Πολιτική κρίση στη Γερμανία: «Να παραιτηθεί ο ...,https://www.naftemporiki.gr/kosmos/1952864/pol...,06/05/2025,14:23,Κόσμος
6,Επιτήρηση και αγγαρεία….,https://www.naftemporiki.gr/opinion/1952849/ep...,06/05/2025,14:21,Απόψεις
7,Ο Κέιν κέρδισε τον πρώτο τίτλο του και παρέδωσ...,https://www.naftemporiki.gr/sports/1952861/o-k...,06/05/2025,14:19,Αθλητικά
8,Στο κυρίως ταμπλό της Ρώμης η Σάκκαρη,https://www.naftemporiki.gr/sports/1952852/sto...,06/05/2025,14:15,Αθλητικά
9,Piraeus Bank reports strong profitability in Q1,https://www.naftemporiki.gr/english/1952854/pi...,06/05/2025,14:15,English
